In [1]:
import os
import numpy as np
import keras.backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import sklearn.svm as svm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import StandardScaler,Normalizer
import pandas as pd
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.utils import to_categorical
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.optimizers import SGD, Adam
from imblearn.datasets import make_imbalance
from imblearn.over_sampling import RandomOverSampler

df = pd.DataFrame(columns=['num_features','fold1','fold2','fold3','fol4'])
#for i in range(1,80):
#for i in range(1,25):
#Accuracy = np.array(i)
def init_weights(m):
    if type(m) == nn.Linear:
        #print(m.weight)
        torch.nn.init.constant_(m.weight,0.1)
        m.bias.data.fill_(0.001)
# for i in range(1,25):
Accuracy = np.array(8)
for fold in range(1,5):
    features = pd.read_csv('ICH_Features.csv',header=0)
    OS_train = pd.read_csv('Fold'+str(fold)+'/Train_dir.txt',header = None, dtype=str)
    OS_train.columns = ['ID','OS']
    OS_valid = pd.read_csv('Fold'+str(fold)+'/Val_Dir.txt',header = None, dtype=str)
    OS_valid.columns = ['ID','OS']
    features['ID']=features['ID'].str.replace('ct1','')
    train = pd.merge(features, OS_train, how='right', on='ID')
    test =  pd.merge(features, OS_valid, how='right', on='ID')
    norm_wihtout = [col for col in train.columns if col not in ['ID','GCS','Onset','OS']]
    #norm_valid = [col for col in test.columns if col not in ['ID','GCS','Onset','OS']]
    scaler = StandardScaler()
    train_ss = scaler.fit_transform(train[norm_wihtout])
    test_ss = scaler.transform(test[norm_wihtout])
    train[norm_wihtout] = train_ss
    test[norm_wihtout] = test_ss
    #train = train.assign(norm_train.values = train_ss)
    col_withoutID = [col for col in train.columns if col not in ['ID','OS']]
    #train_X , train_y = make_imbalance(train[col_withoutID],train['OS'].values.astype(int),sampling_strategy={0:50 , 1:40 },random_state=42)
    ros = RandomOverSampler(random_state=42)
    X_res, y_res = ros.fit_resample(train[col_withoutID], train['OS'].values.astype(int))
    train_X = pd.DataFrame(X_res,columns = col_withoutID)
    train_y = pd.DataFrame(y_res, columns = ['OS'])
    num_features = 8
    estimator = svm.SVC(kernel="linear", C=1)
    print(num_features)
    rfe=RFE(estimator,  n_features_to_select=num_features,step=1)
    rfe.fit(train_X,train_y)
    ranking_RFE=rfe.ranking_
    indices=np.where(ranking_RFE==1)
    indices = list(indices[0])
    data_RFE=train_X.iloc[:,indices]
    valid_RFE = test[col_withoutID].iloc[:,indices]
    print(data_RFE.columns)
    #y_train = to_categorical(train_y.values.astype(int))
    #y_test = to_categorical(test['OS'])
#     lb = preprocessing.LabelBinarizer()
#     lb.fit(train['OS'].values.astype(int))
#     y_test=lb.transform(test['OS'].values.astype(int))
#     y_train=lb.transform(train['OS'].values.astype(int))
#         model = Sequential()
#         model.add(Dense(29, input_dim=num_features, activation='relu',kernel_initializer = keras.initializers.Constant(value=0.01)))
#         model.add(Dense(2,kernel_initializer = keras.initializers.Constant(value=0.01)))
#         model.add(Activation("softmax"))
    opt = Adam(lr=0.000001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model = Sequential()
    #Hidden Layer
    model.add(Dense(9, activation='relu', kernel_initializer=keras.initializers.Constant(value=0.01), input_dim=num_features))
    #Output layer
    model.add(Dense(1, activation='sigmoid', kernel_initializer=keras.initializers.Constant(value=0.01)))
    #model.add(Activation("softmax"))
    # Compile model
    #print(train_y.values.astype(int))
    model.compile(optimizer=opt,loss='binary_crossentropy', metrics =['accuracy'])
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
    mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
    model.fit(data_RFE,train_y.values.astype(int),validation_data=(valid_RFE, test['OS'].values.astype(int)),epochs =10000, verbose = 0, callbacks=[es])
    #saved_model = load_model('best_model.h5')
    # evaluate the model
    _, train_acc = model.evaluate(data_RFE, train_y.values.astype(int), verbose=0)
    _, test_acc = model.evaluate(valid_RFE, test['OS'].values.astype(int), verbose=0)
    print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
    Y_pred=model.predict_classes(valid_RFE)
    #print(Y_pred)
    #print(test['OS'].values.astype(int))
#         #print(y_test)http://localhost:8888/notebooks/ICH/ANN.ipynb#
#         acc=metrics.accuracy_score(test['OS'].values.astype(int),Y_pred)
#         print("accuracy score = "+str(acc)) 
    Accuracy = np.append(Accuracy,test_acc)
Accuracy = pd.DataFrame(data = Accuracy.reshape(1,5),columns = df.columns)
df = df.append(Accuracy)
#del Accuracy

Using TensorFlow backend.


8
Index(['Age', 'Centroid1', 'FirstAxis3', 'SecondAxis1', 'ThirdAxis1',
       'ThirdAxis2', 'kurtosis', 'entropy'],
      dtype='object')
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/utils/validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 00541: early stopping
Train: 0.585, Test: 0.600
8
Index(['FirstAxis3', 'SecondAxis1', 'ThirdAxis1', 'Eigen2', 'bb3', 'extent',
       'diameter', 'f6'],
      dtype='object')


/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/utils/validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 00740: early stopping
Train: 0.521, Test: 0.400
8
Index(['Age', 'MinorAxisLength', 'FirstAxis3', 'SecondAxis1', 'SecondAxis2',
       'ThirdAxis1', 'kurtosis', 'histogram'],
      dtype='object')


/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/utils/validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 00201: early stopping
Train: 0.553, Test: 0.500
8
Index(['Age', 'FirstAxis2', 'FirstAxis3', 'SecondAxis1', 'ThirdAxis1',
       'kurtosis', 'diameter', 'f1'],
      dtype='object')


/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/utils/validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 03964: early stopping
Train: 0.681, Test: 0.600


In [ ]:
train['OS'].values

In [ ]:
df.to_csv('ANN_4_accuracy.csv')

In [ ]:
import shap
print(data_RFE.shape[0])
background = data_RFE.columns[np.random.choice(data_RFE.shape[0], 59, replace=False)]

# explain predictions of the model on three images
e = shap.DeepExplainer(model, background)
# ...or pass tensors directly
# e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)
shap_values = e.shap_values(valid_RFE[1:5])

In [ ]:
df =pd.concat([df,df.iloc[:,1:].mean(axis=1)],axis = 1)
df.columns = ['num_features','fold1','fold2','fold3','fol4','Average']

In [ ]:
df.to_csv('LR_accuracy.csv')

In [ ]:
from scipy.stats import pearsonr
corr, _ = pearsonr(train['kurtosis'].astype(int), train['OS'].astype(int))
print('Pearsons correlation: %.3f' % corr)

In [ ]:
from scipy.stats import pearsonr
corr, _ = pearsonr(train['SecondAxis2'].astype(int), train['OS'].astype(int))
print('Pearsons correlation: %.3f' % corr)

In [ ]:
corr, _ = pearsonr(train['ThirdAxis1'].astype(int), train['OS'].astype(int))
print('Pearsons correlation: %.3f' % corr)

In [ ]:
corr, _ = pearsonr(train['Age'].astype(int), train['OS'].astype(int))
print('Pearsons correlation: %.3f' % corr)

In [ ]:
import shap
import sklearn.svm as svm
 


In [ ]:
print(Y_pred)
print(test['OS'].values)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(test['OS'].values,Y_pred)
# FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)  
# FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
# TP = np.diag(confusion_matrix)
# TN = confusion_matrix.sum() - (FP + FN + TP)

TN,FP,FN,TP = confusion_matrix.ravel()
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
#Precision
PPV = TP/(TP+FP)

print('Sensitivity ='+ str(TPR))
print('Specificity = ' +str(TNR))
print('Precision = '+str(PPV))

In [ ]:
print(confusion_matrix)

In [ ]:
print(FP)

In [ ]:

explainer = shap.LinearExplainer(svm, data_RFE, feature_dependence="independent")
shap_values = explainer.shap_values(data_RFE)
shap.initjs()
# plot the SHAP values for the Setosa output of the first instance
shap.summary_plot(shap_values, data_RFE.iloc[:,:], feature_names=data_RFE.columns,show=True)
